In [4]:

table = client.get_table("patents-public-data.patents.publications")

print("Total columns:", len(table.schema))
for field in table.schema:
    print(field.name, field.field_type, field.mode)

Total columns: 37
publication_number STRING NULLABLE
application_number STRING NULLABLE
country_code STRING NULLABLE
kind_code STRING NULLABLE
application_kind STRING NULLABLE
application_number_formatted STRING NULLABLE
pct_number STRING NULLABLE
family_id STRING NULLABLE
spif_publication_number STRING NULLABLE
spif_application_number STRING NULLABLE
title_localized RECORD REPEATED
abstract_localized RECORD REPEATED
claims_localized RECORD REPEATED
claims_localized_html RECORD REPEATED
description_localized RECORD REPEATED
description_localized_html RECORD REPEATED
publication_date INTEGER NULLABLE
filing_date INTEGER NULLABLE
grant_date INTEGER NULLABLE
priority_date INTEGER NULLABLE
priority_claim RECORD REPEATED
inventor STRING REPEATED
inventor_harmonized RECORD REPEATED
assignee STRING REPEATED
assignee_harmonized RECORD REPEATED
examiner RECORD REPEATED
uspc RECORD REPEATED
ipc RECORD REPEATED
cpc RECORD REPEATED
fi RECORD REPEATED
fterm RECORD REPEATED
locarno RECORD REPEATED
c

In [15]:
from google.cloud import bigquery
import csv

client = bigquery.Client(project="llm-patent-project")

query = """
SELECT
  publication_number,
  title_localized,
  abstract_localized,
  claims_localized,
  claims_localized_html
FROM `patents-public-data.patents.publications`
WHERE publication_date = 20230103
  AND ARRAY_LENGTH(title_localized) > 0
  AND ARRAY_LENGTH(abstract_localized) > 0
  AND ARRAY_LENGTH(claims_localized) > 0
LIMIT 50;
"""

job = client.query(query)

# Open CSV file for writing
with open("patents_output.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    
    # Write header
    writer.writerow([
        "publication_number",
        "title_localized",
        "abstract_localized",
        "claims_localized",
        "claims_localized_html"
    ])
    
    # Write rows
    for row in job:
        writer.writerow([
            row.publication_number,
            row.title_localized,
            row.abstract_localized,
            row.claims_localized,
            row.claims_localized_html
        ])

print("Saved to patents_output.csv")


/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Saved to patents_output.csv


In [1]:
from google.cloud import bigquery
import csv

client = bigquery.Client(project="llm-patent-project")

query = """
SELECT
  publication_number,
  publication_date,
  ARRAY_LENGTH(claims_localized) AS n_claims,
  title_localized
FROM `patents-public-data.patents.publications`
WHERE publication_date = 20230103
  AND ARRAY_LENGTH(claims_localized) IS NOT NULL
ORDER BY n_claims ASC
LIMIT 200;
"""

job = client.query(query)

with open("smallest_claims_patents.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["publication_number", "publication_date", "n_claims", "title_localized"])

    for row in job:
        writer.writerow([
            row.publication_number,
            row.publication_date,
            row.n_claims,
            row.title_localized
        ])

print("Saved to smallest_claims_patents.csv")


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.7) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Saved to smallest_claims_patents.csv


In [2]:
from google.cloud import bigquery
import csv
import heapq

client = bigquery.Client(project="llm-patent-project")

YEARS = [2021, 2022, 2023, 2024]
TOP_K = 100   # how many smallest overall you want

global_heap = []   # min-heap to track global smallest claims

for year in YEARS:
    print(f"Mining year {year} cheaply...")

    query = f"""
    SELECT
      publication_number,
      publication_date,
      ARRAY_LENGTH(claims_localized) AS n_claims,
      title_localized,
      abstract_localized,
      claims_localized,
      claims_localized_html
    FROM `patents-public-data.patents.publications`
    WHERE publication_date BETWEEN {year}0101 AND {year}1231
      AND country_code = "US"
      AND ARRAY_LENGTH(title_localized) > 0
      AND ARRAY_LENGTH(abstract_localized) > 0
      AND ARRAY_LENGTH(claims_localized) > 0
      AND ARRAY_LENGTH(claims_localized_html) > 0
    ORDER BY n_claims ASC
    LIMIT 200;
    """

    job = client.query(query)

    for row in job:
        record = (
            row.n_claims,                       # heap key
            row.publication_number,
            row.publication_date,
            row.title_localized,
            row.abstract_localized,
            row.claims_localized,
            row.claims_localized_html
        )

        if len(global_heap) < TOP_K:
            heapq.heappush(global_heap, record)
        else:
            heapq.heappushpop(global_heap, record)

# Sort final 100 by smallest claim count
global_heap.sort(key=lambda x: x[0])

# Save final result
with open("us_smallest_claims_2021_2024_CHEAP.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "n_claims",
        "publication_number",
        "publication_date",
        "title_localized",
        "abstract_localized",
        "claims_localized",
        "claims_localized_html"
    ])

    for row in global_heap:
        writer.writerow(row)

print("✅ Saved to us_smallest_claims_2021_2024_CHEAP.csv")


/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Mining year 2021 cheaply...
Mining year 2022 cheaply...
Mining year 2023 cheaply...
Mining year 2024 cheaply...
✅ Saved to us_smallest_claims_2021_2024_CHEAP.csv


In [3]:
from google.cloud import bigquery
import csv
import heapq

client = bigquery.Client(project="llm-patent-project")

YEARS = [1985, 1986, 1987, 1988, 1989, 1990]
TOP_K = 500   # desired number of global smallest-claim patents

# Use a max-heap (largest on top) so we keep the smallest K globally
global_heap = []   # (negative_n_claims, ...)

for year in YEARS:
    print(f"Mining year {year} cheaply...")

    query = f"""
    SELECT
      publication_number,
      publication_date,
      ARRAY_LENGTH(claims_localized) AS n_claims,
      title_localized,
      abstract_localized,
      claims_localized,
      claims_localized_html
    FROM `patents-public-data.patents.publications`
    WHERE publication_date BETWEEN {year}0101 AND {year}1231
      AND country_code = "US"
      AND ARRAY_LENGTH(title_localized) > 0
      AND ARRAY_LENGTH(abstract_localized) > 0
      AND ARRAY_LENGTH(claims_localized) > 0
      AND ARRAY_LENGTH(claims_localized_html) > 0
    ORDER BY n_claims ASC
    LIMIT 1000;        -- pull more than we need per year
    """

    job = client.query(query)

    for row in job:
        key = -row.n_claims  # max-heap by negative so smallest is kept

        record = (
            key,
            row.n_claims,
            row.publication_number,
            row.publication_date,
            row.title_localized,
            row.abstract_localized,
            row.claims_localized,
            row.claims_localized_html
        )

        if len(global_heap) < TOP_K:
            heapq.heappush(global_heap, record)
        else:
            # push new → pop largest; ensures we keep the smallest 500
            heapq.heappushpop(global_heap, record)

# Convert heap back to sorted list by claim count
final_rows = sorted(global_heap, key=lambda x: x[1])

# Save final dataset
output_file = "us_smallest_claims_1985_1990_top500.csv"
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "n_claims",
        "publication_number",
        "publication_date",
        "title_localized",
        "abstract_localized",
        "claims_localized",
        "claims_localized_html"
    ])

    for rec in final_rows:
        (_, n_claims, pubnum, pubdate, title, abstract, claims, claims_html) = rec
        writer.writerow([
            n_claims,
            pubnum,
            pubdate,
            title,
            abstract,
            claims,
            claims_html
        ])

print(f"✅ DONE! Saved 500 smallest-claim US patents (1985–1990) → {output_file}")


/Users/emilylugos/opt/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Mining year 1985 cheaply...
Mining year 1986 cheaply...
Mining year 1987 cheaply...
Mining year 1988 cheaply...
Mining year 1989 cheaply...
Mining year 1990 cheaply...
✅ DONE! Saved 500 smallest-claim US patents (1985–1990) → us_smallest_claims_1985_1990_top500.csv
